<a href="https://colab.research.google.com/github/HmxaaK/Forecasting-Macroeconomic-Variables-through-Dynamic-Causal-Analysis/blob/main/Macroeconomics_Indicator_Forecasting_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from statsmodels.tsa import ardl
#Libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.ardl import ARDL,ardl_select_order

#Dataset
data = pd.read_csv('GDP.csv', parse_dates=['observation_date'],
                   index_col='observation_date')

#ADF test
def adf_test(data_set):
  non_stationary_vars = []
  for var in list(data.columns):
    adf = adfuller(data_set[var])
    if adf[1] > 0.10:
      non_stationary_vars.append(var)
  return non_stationary_vars
non_stationary_vars = adf_test(data)
print ('Non-Stationary Variable')
for var in non_stationary_vars:
  print (var)

#Log Differencing
log_GDP = np.log(data['GDPC1'])
Y = log_GDP - log_GDP.shift(1)
Y = Y.dropna()
log_Xs = np.log(data[non_stationary_vars[1:]])
log_diff_Xs = log_Xs - log_Xs.shift(1)
X = pd.concat([log_diff_Xs,data['UMCSENT']],axis=1).dropna()

model = ARDL(endog=Y,lags=1,exog=X,order=1,causal=True).fit()
print (model.summary())

Non-Stationary Variable
GDPC1
FEDFUND
DSPIC96
FGEXPND
EXPGSC1
                                ARDL Model Results                                
Dep. Variable:                      GDPC1   No. Observations:                  262
Model:             ARDL(1, 1, 1, 1, 1, 1)   Log Likelihood                 840.346
Method:                   Conditional MLE   S.D. of innovations              0.010
Date:                    Fri, 14 Nov 2025   AIC                          -1664.692
Time:                            12:50:27   BIC                          -1636.175
Sample:                        04-01-1960   HQIC                         -1653.229
                             - 04-01-2025                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0053      0.004     -1.212      0.227      -0.014       0.003
GDPC1.L1       0.1829      0.079     

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)
